Nesta fase, irei iniciar a etapa de experimentação, onde irei testar uma série 
de modelos de classificação para encontrar o que melhor se adequa ao problema.
Vale ressaltar que apenas algumas colunas serão utilizadas para a criação dos
modelos, pois algumas só são obtidas ao fim do processo do SISU, como é o caso
da coluna "NOTA_CORTE" e "CLASSIFICACAO". Outras colunas como códigos das IES
e cursos também não serão utilizadas, pois não são relevantes para o problema.
Ao final, além do modelo, um conjunto de dados no formato .db será gerado para
ser consumido pelo app final. Segue abaixo as colunas que serão utilizadas:

- Modelo: IES, UF_CAMPUS, MUNICIPIO_CAMPUS, NOME_CURSO, GRAU, TURNO, 
TIPO_MOD_CONCORRENCIA, QT_VAGAS_CONCORRENCIA, PERCENTUAL_BONUS, PESO_L, PESO_CH,
PESO_CN, PESO_M, PESO_R, NOTA_MINIMA_L, NOTA_MINIMA_CH, NOTA_MINIMA_CN, 
NOTA_MINIMA_M, NOTA_MINIMA_R, MEDIA_MINIMA, OPCAO, NOTA_L, NOTA_CH, NOTA_CN, 
NOTA_M, NOTA_R, NOTA_L_COM_PESO, NOTA_CH_COM_PESO, NOTA_CN_COM_PESO, 
NOTA_M_COM_PESO, NOTA_R_COM_PESO, NOTA_CANDIDATO e APROVADO.

Vale ressaltar que parte das informações que serão utilizadas no Web App serão 
buscadas nos dados do SISU, como é o caso da QT_VAGAS_CONCORRENCIA, que é um 
valor que a universidade define para cada curso e não o usuário. Outras serão 
calculadas manualmente, como no caso das notas com peso.


In [1]:
import mlflow
import pandas as pd
import category_encoders as ce

# Preprocessing & Models
from pycaret.classification import *
from sklearn.model_selection import train_test_split


# Métricas
from sklearn.metrics import log_loss

In [2]:
# Lendo os dados
dados_sisu = pd.read_parquet('../data/processed/dados_transformados.parquet/')

In [3]:
# Definindo as colunas que serão utilizadas para treinar o modelo
colunas_para_buscar = ['IES', 'UF_CAMPUS', 'MUNICIPIO_CAMPUS', 'NOME_CURSO', 
                       'GRAU', 'TURNO', 'TIPO_MOD_CONCORRENCIA', 
                       'QT_VAGAS_CONCORRENCIA', 'PERCENTUAL_BONUS', 'PESO_L', 
                       'PESO_CH', 'PESO_CN', 'PESO_M', 'PESO_R', 
                       'NOTA_MINIMA_L', 'NOTA_MINIMA_CH', 'NOTA_MINIMA_CN', 
                       'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA', 
                       'OPCAO', 'NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 
                       'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO', 
                       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
                       'NOTA_CANDIDATO', 'APROVADO']

In [4]:
# Filtrando as colunas
dados_sisu = dados_sisu[colunas_para_buscar]

In [5]:
# Verificando o balanceamento da variável alvo
dados_sisu['APROVADO'].value_counts(normalize=True)

N    0.898245
S    0.101755
Name: APROVADO, dtype: float64

In [6]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

In [7]:
# Dividindo os dados em variaveis explicativas e variavel alvo
x = dados_sisu.drop(columns=['APROVADO'])
y = dados_sisu['APROVADO'].map({'S': 1, 'N': 0})

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.45, random_state=42, stratify=y)

# Dividindo os dados em teste e calibração
x_teste, x_calibracao, y_teste, y_calibracao = train_test_split(x_teste, y_teste, test_size=0.5, random_state=42, stratify=y_teste)

In [8]:
# Criando o setup do experimento para modelos lineares sem transformers
setup_modelos_lineares_ = setup(data=pd.concat([x_treino, y_treino], axis = 1), 
                                target='APROVADO', normalize=True, n_jobs=3,
                                normalize_method='zscore', log_experiment=True, 
                                experiment_name='modelos_lineares_sem_transformers',
                                session_id=523, fold=5)

# Adicionando a log loss como métrica
add_metric('log_loss', 'LogLoss', log_loss, greater_is_better=False, target='pred_proba')

,Description,Value
0,Session id,523
1,Target,APROVADO
2,Target type,Binary
3,Original data shape,"(1117970, 33)"
4,Transformed data shape,"(1117970, 100)"
5,Transformed train set shape,"(782579, 100)"
6,Transformed test set shape,"(335391, 100)"
7,Ordinal features,1
8,Numeric features,19
9,Categorical features,13


2023/12/04 20:09:50 INFO mlflow.tracking.fluent: Experiment with name 'modelos_lineares_sem_transformers' does not exist. Creating a new experiment.


Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: log_loss, dtype: object

In [9]:
# Comparando apenas os modelos sensiveis a escala
compare_models(sort='LogLoss', include=['lr', 'svm', 'mlp', 'lda', 'qda', 'ridge'], )

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:09:51
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
svm,SVM - Linear Kernel,0.8985,0.0000,0.0072,0.6136,0.0143,0.0119,0.0577,0.0000,6.3440
ridge,Ridge Classifier,0.8995,0.0000,0.0219,0.7056,0.0424,0.0365,0.1123,0.0000,4.7540
mlp,MLP Classifier,0.9233,0.9393,0.4448,0.6919,0.5413,0.5016,0.5163,0.1752,148.5760
lr,Logistic Regression,0.9066,0.8882,0.2459,0.5997,0.3488,0.3078,0.3437,0.2252,10.0780
lda,Linear Discriminant Analysis,0.9039,0.8837,0.2234,0.5712,0.3212,0.2800,0.3161,0.2319,8.2100
qda,Quadratic Discriminant Analysis,0.5654,0.5705,0.5537,0.1272,0.2057,0.0490,0.0745,13.0275,8.0180


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=3, penalty='l2',
              power_t=0.5, random_state=523, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)